# If you do not have the dependencies installed, run these.

In [ ]:
%pip install pandas
%pip install sqlalchemy
%pip install PyMySQL

In [1]:
import pandas as pd
import numpy as np
import json

# Here we read in data downloaded as CSV form from ``cdli_db``.

In [2]:
pubs = pd.read_csv("publications.csv", index_col = "id")
artifs_pubs = pd.read_csv("artifacts_publications.csv")
authors_pubs = pd.read_csv("authors_publications.csv")
abbrev = pd.read_csv("abbreviations.csv")

C:\Users\Circle\AppData\Local\Temp/ipykernel_53976/2139748124.py:1: DtypeWarning: Columns (2,3,5,7,8,11,12,17,19,20,21,22,23,25) have mixed types. Specify dtype option on import or set low_memory=False.
  pubs = pd.read_csv("publications.csv", index_col = "id")
C:\Users\Circle\AppData\Local\Temp/ipykernel_53976/2139748124.py:2: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  artifs_pubs = pd.read_csv("artifacts_publications.csv")


If you would prefer to read from the SQL (framework needs to be up and running) you can run these lines.

In [2]:
from sqlalchemy import create_engine
db_server = "mariadb+mariadbconnector://root@127.0.0.1:23306/cdli_db"
engine = create_engine(db_server)
connection = engine.connect()

In [3]:
pubs = pd.read_sql("SELECT * FROM publications", connection, index_col = "id")
artifs_pubs = pd.read_sql("SELECT * FROM entities_publications WHERE table_name = 'artifacts'", connection)
authors_pubs = pd.read_sql("SELECT * FROM authors_publications", connection)
abbrev = pd.read_sql("SELECT * FROM abbreviations", connection)

# Step 1: Identify subsets of publications that may be curated.

A lot of fields have the form like "ATU 3, pl. 036, W 12139", where "ATU 3" is an assyriological abbreviation and what goes
after the comma is exact_reference.

ATU 3 is the **real** designation and the pl. 036, W 12139 is the exact reference.

So first we want to pull out the real designation. The method here is to find the first occurrence of the pattern ``\w+ \d+``,
which is basically a bunch of characters followed by a bunch of numbers. The matching occurence is named "designation_first" and inserted
as a column. 

Then we pick all rows where there is a numeric character in "designation_first".

In [4]:
des_first = pubs["designation"].fillna("").str.findall(r'\w+ \d+').str[0].fillna("")
# des_first = pubs["designation"].fillna("").str.split(",", n = 1, expand = True)[0]
pubs["designation_first"] = des_first
mergeable = pubs[pubs["designation_first"].str.match(".*\d.*")]

Then we check if these abbreviations exist in the abbreviations table. We split the "designation_first" column by space and take
what goes before it. Then we check the abbreviations against the existing abbreviations, which isolates those that exist in the
abbreviations.

In [5]:
mergeable_abbrevs = mergeable["designation_first"].str.split(" ", n = 1, expand = True)[0].to_numpy()
mergeable.loc[:, "designation_abbrev"] = mergeable_abbrevs

all_abbrevs = abbrev["abbreviation"].str.replace("\s\(aka.*\)", "", regex = True).str.strip()
mergeable = mergeable[mergeable["designation_abbrev"].isin(all_abbrevs)]
mergeable.head()

d:\Programming\Anaconda\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
d:\Programming\Anaconda\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,designation,bibtexkey,year,entry_type_id,address,annote,book_title,chapter,crossref,edition,...,publisher,school,title,volume,series,oclc,accepted_by,accepted,designation_first,designation_abbrev
id,,,,,,,,,,,,,,,,,,,,,
468,CUSAS 35,468,2017,NaN,None,None,None,None,None,None,...,None,None,Sumerian Administrative and Legal Documents ca...,None,None,None,820,1,CUSAS 35,CUSAS
60627,OSP 2,60627,1987,2.0,None,None,None,None,None,None,...,Museum Tusculanum Press,None,Old Sumerian and Old Akkadian Texts in Philade...,None,Carsten Niebuhr Institute Publications,None,820,1,OSP 2,OSP
69162,OIP 011,69162,1929,2.0,Chicago,None,None,None,None,None,...,The University of Chicago Press,None,Sumerian Lexical Texts from the Temple School ...,1,Oriental Institute Publications,None,820,1,OIP 011,OIP
98673,"RSO 90, 030",98673,2017,1.0,None,None,None,None,None,None,...,None,None,A round tablet concerning gardens from Ur III ...,90,None,None,820,1,RSO 90,RSO
117910,OIP 1111,117910,1993,NaN,None,None,None,None,None,None,...,None,None,Kassite Buildings in Area WC-1,None,None,None,820,1,OIP 1111,OIP


We also remove entries with only 1 unique "designation_first". There's no point for merging them at this point.

In [6]:
def filter_func(df):
    return df.shape[0] > 1

mergeable = mergeable.groupby("designation_first").filter(filter_func)
mergeable

,designation,bibtexkey,year,entry_type_id,address,annote,book_title,chapter,crossref,edition,...,publisher,school,title,volume,series,oclc,accepted_by,accepted,designation_first,designation_abbrev
id,,,,,,,,,,,,,,,,,,,,,
468,CUSAS 35,468,2017,NaN,None,None,None,None,None,None,...,None,None,Sumerian Administrative and Legal Documents ca...,None,None,None,820,1,CUSAS 35,CUSAS
60627,OSP 2,60627,1987,2.0,None,None,None,None,None,None,...,Museum Tusculanum Press,None,Old Sumerian and Old Akkadian Texts in Philade...,None,Carsten Niebuhr Institute Publications,None,820,1,OSP 2,OSP
69162,OIP 011,69162,1929,2.0,Chicago,None,None,None,None,None,...,The University of Chicago Press,None,Sumerian Lexical Texts from the Temple School ...,1,Oriental Institute Publications,None,820,1,OIP 011,OIP
98673,"RSO 90, 030",98673,2017,1.0,None,None,None,None,None,None,...,None,None,A round tablet concerning gardens from Ur III ...,90,None,None,820,1,RSO 90,RSO
252543,"Rattenborg, Rune, NABU 2014/010",252543,2014,1.0,None,None,None,None,None,None,...,None,None,Further on NABU 2001/7: an administrative tabl...,None,None,None,820,1,NABU 2014,NABU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1484955,"ASJ 16, 310-311",1484955,None,NaN,None,None,None,None,None,None,...,None,None,None,None,None,None,820,1,ASJ 16,ASJ
1484956,"OrAnt 17, 060",1484956,None,NaN,None,None,None,None,None,None,...,None,None,None,None,None,None,820,1,OrAnt 17,OrAnt
1484959,"W. Heimpel, JAOS 111 (1991) 380-381",1484959,None,NaN,None,None,None,None,None,None,...,None,None,None,None,None,None,820,1,JAOS 111,JAOS


Here we do a final round of filtering where we only select those pubs that has an author attached.

In [7]:
mergeable = mergeable[mergeable.index.isin(authors_pubs["publication_id"].unique())]

In [19]:
mergeable.to_csv("before_merge.csv")

In [20]:
unchanged = pubs[~pubs.index.isin(mergeable.index)]
unchanged

,designation,bibtexkey,year,entry_type_id,address,annote,book_title,chapter,crossref,edition,...,pages,publisher,school,title,volume,series,oclc,accepted_by,accepted,designation_first
id,,,,,,,,,,,,,,,,,,,,,
1,Bulla,1,1981,7.0,Winona Lake,None,In Honor of Ernest R. Lacheman on His Seventy-...,None,None,None,...,1-9,Eisenbrauns,None,Notes on a Pair of Matching Texts: A Shepherd'...,1,SCCNH,None,820,1,
2,AbuschSchwemer2011,2,None,2.0,,,Corpus of Mesopotamian Anti-Witchcraft Rituals...,,,,...,,Brill: Leiden,,,8,Ancient Magic and Divination,None,820,1,
3,Alizadeh2008,3,None,2.0,,,Chogha Mish II: The Development of a Prehistor...,,,,...,,The Oriental Institute: Chicago,,,130,OIP,None,820,1,
4,Allred2006,4,None,15.0,,,Cooks and Kitchens: Centralized Food Productio...,,,,...,,,Johns Hopkins University,,,,None,820,1,
5,Girsu Labor,5,2008,7.0,None,None,On the Third Dynasty of Ur: Studies in Honor o...,None,None,None,...,11-19,The American Schools of Oriental Research,None,Labor Assignments from the City of Girsu,1,The Journal of Cuneiform Studies Supplemental ...,None,820,1,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1484958,"Boiy, 2005, 109",1484958,None,NaN,None,None,None,None,None,None,...,None,None,None,None,None,None,None,820,1,
1484959,"W. Heimpel, JAOS 111 (1991) 380-381",1484959,None,NaN,None,None,None,None,None,None,...,None,None,None,None,None,None,None,820,1,JAOS 111
1484960,"W. Sallaberger, ZA 82 (1992) 131-137",1484960,None,NaN,None,None,None,None,None,None,...,None,None,None,None,None,None,None,820,1,ZA 82


# Step 2: Try to merge some of them.

In [21]:
# This part is added after Adam's proofreading

# Fixed some of the failures of mis-identifying designation_first
# The program will go through every element in ``find_and_fix``, for every element...
# The rows whose designation contains that element will have their designation_first field will be changed
find_and_fix = ["AOS 32", "Arch. 79", "AAT", "BBR", "BTT", "BWL", "CHJ", "CIRPL",
    "Dreams", "EDATS", "EEN", "FAOS 19", "Fs Kraus", "Fs Leichty",
    "Fs Lenoble", "Fs Matous", "Fs Owen", "Fs Pettinato", "Fs Sachs",
    "Fs Sigrist", "Fs Steve", "GCCI", "ITT 1", "ITT 2", "ITT 5", "KAR",
    "KAV", "Kish", "LKU", "MAD 1", "MCT", "MDP 31", "MEE 03", "MSKH",
    "MSL 04", "MSL 10", "MSL 11", "MSL 13", "MSL 14", "MSL 17", "MSL 4",
    "MSL 5", "MSL 6", "MSL 7", "MSL 8/1", "MSL 8/2", "MSL 9", "MSL SS",
    "MVAG 8/5", "NATN", "NFT", "Nisaba 04", "Nisaba 05", "Nisaba 12",
    "NRV", "OIP 138", "OrSP 06", "Phoenix 23", "Phoenix Ancient Art",
    "Proverbs", "PRT", "RIMA 1", "RIMA 2", "RIMA 3", "RIMB 1", "RIMB 2",
    "RIME 1", "RIME 2", "RIME 3", "RIME 4", "RINAP 1", "RINAP 3",
    "RINAP 4", "RINAP 5", "RMA", "RSO 05", "RT 22", "SBH", "SET", "SLFN",
    "TCND", "TCNU", "TJA", "TJDB", "TMH 2-3", "TMN", "UCP 09-02", "UNT",
    "VS 01", "YNER 4"]

for e in find_and_fix:
    mergeable.loc[mergeable["designation"].str.contains(e), "designation_first"] = e

d:\Programming\Anaconda\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


# Here's the code that generates all indices to be merged

In [22]:
replacement_indices = []
for abbrev_name in mergeable["designation_first"].unique():

    # Has same name
    has_same_name = mergeable["designation_first"] == abbrev_name

    # Find all entries with that abbreviation name
    subset = mergeable[has_same_name]

    # Skip if only 1
    if subset.shape[0] == 1:
        continue

    # Split into entries to be replaced (to_replace), and what they will be changed to (replace_with)
    # Prioritize those with a book title or a title as the "representative"
    represent = subset[~subset["book_title"].isna() | ~subset["title"].isna()]

    # If no "representative" exist, just choose the first one
    if represent.shape[0] < 1:
        represent = subset.iloc[0:1]
    represent = represent.iloc[0:1]
    assert represent.shape[0] > 0
    
    # Fetch the id these publications will be changed to
    replace_id = represent.index.item()

    # Fetch the id of the publications to be replaced
    ids_to_replace = subset.index[~subset.index.isin([replace_id])]

    # Append the indices to replace to the correct places
    replacement_indices.append([ids_to_replace, replace_id])

In [23]:
for e in replacement_indices:
    e[0] = list(e[0])

json_dump = []
for e in replacement_indices:
    json_dump.append({"merge" : e[0],"to" : e[1]})
    
with open("merge_metadata.json", "w") as f:
    f.write(json.dumps(json_dump, indent = 4))

# Step 1 & 2 Alternative: If you have the .json files, you can simply run the follwing cells, and then read in the merge_metadata.json to get the ids_to_replace

In [24]:
mergeable = pd.read_csv("before_merge.csv", index_col = "id")

C:\Users\Circle\AppData\Local\Temp/ipykernel_53976/1108354287.py:1: DtypeWarning: Columns (5,7,12,19,20,22,25) have mixed types. Specify dtype option on import or set low_memory=False.
  mergeable = pd.read_csv("before_merge.csv", index_col = "id")


In [25]:
with open("merge_metadata.json", "r") as f:
    merge_metadata = json.load(f)

In [26]:
replacement_indices = []
for e in merge_metadata:
    replacement_indices.append([e["merge"], e["to"]])

# Step 3: Actually merge the publications and update the join tables

## For the next cell, we just define the methods used for merging

In [25]:
# Old version, no ML involved
# This function cleans the exact_reference entry and does the appropriate merging.

def clean_entries(indices_involved):
    
    curr_designation = pd.Series(mergeable.loc[indices_involved, "designation"])
    merged_designation = mergeable.loc[indices_involved, "designation_first"].iloc[0]

    year_pattern = r'(?:\(|\[)(\d{4})(?:\)|\])'

    # Extract all of the information that we want to extract
    year_info = curr_designation.str.findall(r'\((\d{4})\)').str[0]
    year_info = year_info[year_info.notna()]

    # Now we start taking the irrelevant information away from the exact reference
    # Here it takes away the author names and the year
    exact_reference = curr_designation.str.replace(merged_designation, "", regex = False)\
        .str.replace(r'(?:[A-Z][a-z]+[,\s&]*|[A-Z]\.),?', "", regex = True) \
        .str.replace(year_pattern, "", regex = True).str.strip('., \n\t')

    # Fill in all relevant information to the corresponding columns here
    mergeable.loc[year_info.index, "year"] = year_info.to_numpy()

    # Update the artifacts_publications table here
    for id_to_update in indices_involved:
        if all(artifs_pubs.loc[artifs_pubs["publication_id"].isin([id_to_update]), "exact_reference"].notna()):
            continue
        artifs_pubs.loc[artifs_pubs["publication_id"].isin([id_to_update]), "exact_reference"] = exact_reference[id_to_update]
    

def replace(ids_to_replace, replace_id):
    if not any([(id in mergeable.index) for id in ids_to_replace]):
        print("Id not found.")
        return
        
    indices_involved = ids_to_replace + [replace_id]
    clean_entries(indices_involved)
    
    # Replace the ids in artifacts_publications here
    for id_to_replace in ids_to_replace:
        artifs_pubs.loc[artifs_pubs["publication_id"].isin([id_to_replace]), "publication_id"] = replace_id
        
    # Update the mergeable table
    mergeable.drop(mergeable.index[mergeable.index.isin(ids_to_replace)], inplace = True)

In [26]:
def store_progress(progress_idx):
    mergeable.to_csv("mergeable_wip.csv")
    artifs_pubs.to_csv("artifs_pubs_wip.csv")
    with open("replace_idx.txt", "w") as f:
        f.write(str(progress_idx))

In [27]:
def retrieve_progress():
    global mergeable, artifs_pubs
    mergeable = pd.read_csv("mergeable_wip.csv", index_col = "id")
    artifs_pubs = pd.read_csv("artifs_pubs_wip.csv")
    prgrs = 0
    with open("replace_idx.txt", "r") as f:
        prgrs = int(f.readline())
    print(f'Progress retrieved on index {prgrs}')
    return prgrs

In [28]:
prgrs = 0

In [113]:
# If there is a progress file, retrieve the progress
prgrs = retrieve_progress()

d:\Programming\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (2,3,5,7,12,19,20,22,25) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Progress retrieved on index 624


d:\Programming\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [29]:
for ids_to_replace, replace_id in replacement_indices[prgrs:]:

    replace(ids_to_replace, replace_id) 
    if prgrs % 100 == 0:
        print(f"{prgrs} / {len(replacement_indices)}")
        store_progress(prgrs)
    prgrs += 1

d:\Programming\Anaconda\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


0 / 1582
100 / 1582
200 / 1582
300 / 1582
400 / 1582
500 / 1582
600 / 1582
700 / 1582
800 / 1582
900 / 1582
1000 / 1582
1100 / 1582
1200 / 1582
1300 / 1582
1400 / 1582
1500 / 1582


In [30]:
mergeable["designation"] = mergeable["designation_first"]

d:\Programming\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# Step 4: There's also some ad-hoc fixing - so we apply some additional manual fixes not accounted for by the program

In [31]:
with open("corrections.json", "r") as f:
    fixes = json.load(f)

In [32]:
mergeable.drop(mergeable.index[mergeable.index.isin(fixes["special"])], inplace = True)
mergeable

,designation,bibtexkey,year,entry_type_id,address,annote,book_title,chapter,crossref,edition,...,publisher,school,title,volume,series,oclc,accepted_by,accepted,designation_first,designation_abbrev
id,,,,,,,,,,,,,,,,,,,,,
60627,OSP 2,60627,1987,2.0,None,None,None,None,None,None,...,Museum Tusculanum Press,None,Old Sumerian and Old Akkadian Texts in Philade...,None,Carsten Niebuhr Institute Publications,None,820,1,OSP 2,OSP
69162,OIP 011,69162,1929,2.0,Chicago,None,None,None,None,None,...,The University of Chicago Press,None,Sumerian Lexical Texts from the Temple School ...,1,Oriental Institute Publications,None,820,1,OIP 011,OIP
98673,RSO 90,98673,2017,1.0,None,None,None,None,None,None,...,None,None,A round tablet concerning gardens from Ur III ...,90,None,None,820,1,RSO 90,RSO
252543,NABU 2014,252543,2014,1.0,None,None,None,None,None,None,...,None,None,Further on NABU 2001/7: an administrative tabl...,None,None,None,820,1,NABU 2014,NABU
410535,NABU 1992,410535,1992,1.0,None,None,None,None,None,None,...,None,None,The continuing story of Sippar-Amnānum = Sippa...,1,None,None,820,1,NABU 1992,NABU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1359385,BiMes 29,1359385,2011,NaN,None,None,None,None,None,None,...,None,None,None,None,None,None,820,1,BiMes 29,BiMes
1359480,JCS 50,1359480,1998,NaN,None,None,None,None,None,None,...,None,None,None,None,None,None,820,1,JCS 50,JCS
1359482,NABU 1999,1359482,1999,NaN,None,None,None,None,None,None,...,None,None,None,None,None,None,820,1,NABU 1999,NABU


In [33]:
for merge_dict in fixes['additional_merge']:
    ids_to_replace, replace_id = merge_dict['merge'], merge_dict['to']
    replace(ids_to_replace, replace_id)

Id not found.
Id not found.
Id not found.
Id not found.
Id not found.
Id not found.
Id not found.
Id not found.
Id not found.
Id not found.
Id not found.
Id not found.
Id not found.
Id not found.
Id not found.


In [50]:
merged_with_join = mergeable.merge(artifs_pubs, how = 'inner', left_index = True, right_on = 'publication_id')

In [51]:
artifacts = merged_with_join["artifact_id"]
pubs = merged_with_join["publication_id"]
exact_ref = merged_with_join["exact_reference"]

merged_with_join = merged_with_join.drop(columns = ["artifact_id", "publication_id", "exact_reference"])
merged_with_join.insert(0, "artifact_id", artifacts)
merged_with_join.insert(0, "publication_id", pubs)
merged_with_join.insert(5, "exact_reference", exact_ref)


In [39]:
mergeable.drop(columns = ["designation_first", "designation_abbrev"], inplace = True)

d:\Programming\Anaconda\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [42]:
final = mergeable.append(unchanged).drop(columns = "designation_first")

In [45]:
artifs_pubs

,id,entity_id,publication_id,exact_reference,publication_type,publication_comments,table_name
0,275152265,286566,98673,030,primary,None,artifacts
1,275171490,332927,117910,"pl. 098, 14 N 229",primary,None,artifacts
2,275183797,349329,117910,"pl. 093, 13 N 126",primary,None,artifacts
3,275183798,349330,117910,"pl. 093, 13 N 127",primary,None,artifacts
4,275183799,349331,117910,"pl. 094, 13 N 131",primary,None,artifacts
...,...,...,...,...,...,...,...
230216,276062484,532443,1359502,None,primary,None,artifacts
230217,276062485,532444,1291026,173-175 (pl. 71-72) TA 2100,primary,None,artifacts
230218,276062486,532445,1291026,173-175 (pl. 71-72) TA 2101,primary,None,artifacts
230219,276062487,532446,1359505,None,primary,None,artifacts


# Now we write the csv to the file

In [52]:
final.to_csv("merged_publications.csv")
artifs_pubs.to_csv("merged_artifacts_publications.csv", index = False)

In [10]:
f = pd.read_csv("merged_publications.csv", index_col = "id")

d:\Programming\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3229: DtypeWarning: Columns (3,5,7,8,11,16,18,19,20,21,22,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
